In [1]:
import sys
from pyspark.context import SparkContext, SparkConf
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions

In [2]:
gc = GlueContext(SparkContext.getOrCreate())
gc.setConf("spark.sql.ansi.enabled","true")
job = Job(gc)
job.init('test')

In [3]:
def read_json(glue_context, path):
    dynamicframe = glue_context.create_dynamic_frame.from_options(
        connection_type='s3',
        connection_options={
            'paths': [path],
            'recurse': True
        },
        format='json'
    )
    return dynamicframe

In [4]:
dyf = read_json(gc, "s3://awsglue-datasets/examples/us-legislators/all/persons.json")
dyf.printSchema()

job.commit()

root
|-- family_name: string
|-- name: string
|-- links: array
|    |-- element: struct
|    |    |-- note: string
|    |    |-- url: string
|-- gender: string
|-- image: string
|-- identifiers: array
|    |-- element: struct
|    |    |-- scheme: string
|    |    |-- identifier: string
|-- other_names: array
|    |-- element: struct
|    |    |-- lang: string
|    |    |-- note: string
|    |    |-- name: string
|-- sort_name: string
|-- images: array
|    |-- element: struct
|    |    |-- url: string
|-- given_name: string
|-- birth_date: string
|-- id: string
|-- contact_details: array
|    |-- element: struct
|    |    |-- type: string
|    |    |-- value: string
|-- death_date: string



In [11]:
# gc.sql()로 spark.sql()과 동일하게 sql작성 가능
test_dict = {
    "choi":20,
    "park":29,
    "cha":30
}
df = gc.createDataFrame(test_dict.items(), ["name", "age"])
df.registerTempTable("test_table")

gc.sql("""
SELECT * 
FROM test_table
""").show()

+----+---+
|name|age|
+----+---+
|choi| 20|
|park| 29|
| cha| 30|
+----+---+



In [12]:
gc.sql("""
SELECT avg(age) as avg_age
FROM test_table
""").show()

+------------------+
|           avg_age|
+------------------+
|26.333333333333332|
+------------------+



In [13]:
# 2번째 cell의 옵션이 현재 true로 되어있어 에러가 남
# false로 수정하면 hive sql로 실행되는 것을 볼수 있음
# gc.setConf("spark.sql.ansi.enabled","true")
gc.sql("SELECT CAST('a' AS INT)").show()

Py4JJavaError: An error occurred while calling o95.showString.
: java.lang.NumberFormatException: invalid input syntax for type numeric: a
	at org.apache.spark.unsafe.types.UTF8String.toIntExact(UTF8String.java:1347)
	at org.apache.spark.sql.catalyst.expressions.CastBase.$anonfun$castToInt$2(Cast.scala:537)
	at org.apache.spark.sql.catalyst.expressions.CastBase.$anonfun$castToInt$2$adapted(Cast.scala:537)
	at org.apache.spark.sql.catalyst.expressions.CastBase.buildCast(Cast.scala:295)
	at org.apache.spark.sql.catalyst.expressions.CastBase.$anonfun$castToInt$1(Cast.scala:537)
	at org.apache.spark.sql.catalyst.expressions.CastBase.nullSafeEval(Cast.scala:840)
	at org.apache.spark.sql.catalyst.expressions.UnaryExpression.eval(Expression.scala:476)
	at org.apache.spark.sql.catalyst.expressions.UnaryExpression.eval(Expression.scala:472)
	at org.apache.spark.sql.catalyst.optimizer.ConstantFolding$$anonfun$apply$1$$anonfun$applyOrElse$1.applyOrElse(expressions.scala:66)
	at org.apache.spark.s

In [16]:
gc.sql("SELECT CAST('a' AS INT)").show()

+--------------+
|CAST(a AS INT)|
+--------------+
|          null|
+--------------+

